In [1]:
import numpy as np
import pandas as pd
import os

# Progress bar
from tqdm import tqdm


import tensorflow as tf
from tensorflow.keras.utils import Sequence

# Preprocessing data

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Layer model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from tensorflow.keras.layers import Embedding, LSTM, add, Concatenate, Reshape, concatenate, Bidirectional


from tensorflow.keras.applications import ResNet50, DenseNet201

# optimizer
from tensorflow.keras.optimizers import Adam

# Callback
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2025-03-03 11:25:31.977938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740981332.057464    8069 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740981332.082980    8069 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 11:25:32.270861: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
labels_csv = pd.read_csv("../kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/results.csv",delimiter='|')

In [3]:
labels_csv.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [4]:
# drop column='image_name'
labels_csv=labels_csv.drop(columns=['image_name'])
# drop column='comment_number'
labels_csv=labels_csv.drop(columns=[' comment_number'])

In [5]:
labels_csv.columns = ['comment']

In [6]:
labels_csv = labels_csv.dropna(subset=['comment'])


In [7]:
labels_csv.head()

,comment
0,Two young guys with shaggy hair look at their...
1,"Two young , White males are outside near many..."
2,Two men in green shirts are standing in a yard .
3,A man in a blue shirt standing in a garden .
4,Two friends enjoy time spent together .


In [8]:
def text_preprocessing(data):
    data['comment'] = data['comment'].apply(lambda x: x.lower())
    data['comment'] = data['comment'].apply(lambda x: x.replace("[^A-Za-z]",""))
    data['comment'] = data['comment'].apply(lambda x: x.replace("\s+"," "))
    data['comment'] = data['comment'].apply(lambda x: " ".join([word for word in x.split() if len(word)>1]))
    data['comment'] = "startseq "+data['comment']+" endseq"
    return data

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_8069/1899435095.py:4: SyntaxWarning: invalid escape sequence '\s'
  data['comment'] = data['comment'].apply(lambda x: x.replace("\s+"," "))


In [9]:
data = text_preprocessing(labels_csv)
captions = labels_csv['comment'].tolist()

In [10]:
captions

['startseq two young guys with shaggy hair look at their hands while hanging out in the yard endseq',
 'startseq two young white males are outside near many bushes endseq',
 'startseq two men in green shirts are standing in yard endseq',
 'startseq man in blue shirt standing in garden endseq',
 'startseq two friends enjoy time spent together endseq',
 'startseq several men in hard hats are operating giant pulley system endseq',
 'startseq workers look down from up above on piece of equipment endseq',
 'startseq two men working on machine wearing hard hats endseq',
 'startseq four men on top of tall structure endseq',
 'startseq three men on large rig endseq',
 'startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq little girl in pink dress going into wooden cabin endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq girl going into wooden building en

In [11]:
# Step 1: Initialize a tokenizer object
tokenizer = Tokenizer()

# Step 2: Fit the tokenizer on the list of text captions
# This builds a vocabulary from the given captions
tokenizer.fit_on_texts(captions)

# Step 3: Get the vocabulary size
# `word_index` is a dictionary mapping words to their unique integer index
# We add 1 because index 0 is reserved for padding
vocab_size = len(tokenizer.word_index) + 1

# Step 4: Determine the maximum length of any caption in the dataset
# Splitting each caption into words and finding the max word count
max_length = max(len(caption.split()) for caption in captions)


# Step 11: Convert a sample caption into a sequence of tokenized words
# `texts_to_sequences()` replaces words with their corresponding integer tokens
tokenized_caption = tokenizer.texts_to_sequences([captions[1]])[0]


In [12]:
tokenized_caption

[1, 12, 21, 20, 708, 13, 54, 79, 194, 1421, 2]

In [13]:
def idx_to_word(integer,tokenizer):
    
    for word, index in tokenizer.word_index.items():
        if index==integer:
            return word
    return None

In [14]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load DenseNet201 model for feature extraction
model = DenseNet201()
fe = Model(inputs=model.input, outputs=model.layers[-2].output)

# Function to extract features from an image
def extract_features(image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))  # Resize image to DenseNet input size
    img = img_to_array(img)
    img = img/255.
    img = np.expand_dims(img,axis=0)
    features = fe.predict(img)
    return features

2025-03-03 11:25:38.466126: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
image_features=extract_features("dog.webp")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


In [16]:
image_features

array([[8.5647378e-05, 4.1139961e-04, 1.1667424e-03, ..., 3.2138574e-01,
        6.2093472e-01, 7.5144254e-02]], dtype=float32)

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
import gradio as gr
import spacy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from nltk.corpus import wordnet as wn
import base64
from pathlib import Path

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Load the trained captioning model
caption_model = load_model('model.h5')

# Predefined motion words
motion_words = {
    "run", "walk", "jump", "fly", "swim", "move", "crawl", "slide",
    "throw", "catch", "push", "pull", "drag", "lift", "climb", "dive",
    "skip", "hop", "roll", "stumble", "fall", "dash", "chase"
}

def get_motion_synonyms():
    """Fetch motion-related synonyms from WordNet."""
    motion_synsets = wn.synset('move.v.01').closure(lambda s: s.hyponyms())
    for synset in motion_synsets:
        motion_words.update(lemma.name().replace("_", " ") for lemma in synset.lemmas())

# Expand motion words with synonyms
get_motion_synonyms()

def detect_motion(text):
    """Detects motion words in a caption, ensuring only verbs are considered."""
    doc = nlp(text)
    for token in doc:
        if token.lemma_ in motion_words and token.pos_ == "VERB":
            return "Motion Detected"
    return "No Motion Detected"

def predict_caption(image_path):
    """Generates caption and detects motion."""
    feature = extract_features(image_path)  
    caption = "startseq"

    # Generate caption
    for _ in range(max_length):
        seq = tokenizer.texts_to_sequences([caption])[0]
        seq = pad_sequences([seq], maxlen=max_length)

        yhat = caption_model.predict([feature, seq], verbose=0)
        yhat = np.argmax(yhat)  

        word = idx_to_word(yhat, tokenizer)
        if word is None or word == 'endseq':
            break

        caption += " " + word

    caption = caption.replace("startseq", "").strip()
    
    # Detect motion
    motion_result = detect_motion(caption)

    return caption, motion_result

# Load animation file
def load_animation():
    """Loads animation (GIF) file for motion detection."""
    try:
        animation_path = Path("running_animation.gif")  # Replace with your animation file
        if not animation_path.exists():
            raise FileNotFoundError(f"Animation file not found: {animation_path}")

        # Encode animation file to base64
        with open(animation_path, "rb") as f:
            animation_base64 = base64.b64encode(f.read()).decode("utf-8")
        return animation_base64
    except Exception as e:
        print(f"Error loading animation: {e}")
        return None

animation_base64 = load_animation()

# Gradio UI with full-screen animation
css = """
.fullscreen-overlay {
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: rgba(0, 0, 0, 0.8);
    display: flex;
    justify-content: center;
    align-items: center;
    z-index: 1000;
    animation: fadeIn 0.5s ease-in-out;
}
.fullscreen-overlay img {
    max-width: 100%;
    max-height: 100%;
}
"""

# JavaScript to show and hide the full-screen overlay
js = """
function showFullscreenAnimation() {
    console.log("Showing full-screen animation...");  // Debugging log
    const overlay = document.getElementById('fullscreen-overlay');
    if (overlay) {
        overlay.style.display = 'flex';
        setTimeout(() => {
            overlay.style.display = 'none';
        }, 3000);  // Hide after 3 seconds
    } else {
        console.error("Full-screen overlay not found!");  // Debugging log
    }
}
"""

# Gradio interface
with gr.Blocks(css=css) as demo:
    gr.Markdown("# 🖼️ Image Motion and Appearance Detector")
    gr.Markdown("Upload an image, and the model will generate a caption. It will also detect if motion is present in the description.")

    # Full-screen overlay for animation
    gr.HTML(f"""
    <div id="fullscreen-overlay" class="fullscreen-overlay" style="display: none;">
        <img src="data:image/gif;base64,{animation_base64}" alt="Motion Animation">
    </div>
    <script>{js}</script>
    """)

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="filepath", label="Upload Image")
            submit_button = gr.Button("Generate Caption", variant="primary")
        
        with gr.Column():
            caption_output = gr.Textbox(label="Image Description", lines=3, placeholder="Generated caption will appear here...")
            motion_output = gr.Textbox(label="Motion Detection Result", lines=2, placeholder="Motion detection result will appear here...")

    def update_ui(image_path):
        caption, motion_result = predict_caption(image_path)
        if motion_result == "Motion Detected" and animation_base64:
            # Trigger the full-screen animation
            return caption, motion_result, gr.HTML("<script>showFullscreenAnimation();</script>")
        else:
            return caption, motion_result, gr.HTML("")

    submit_button.click(
        fn=update_ui,
        inputs=image_input,
        outputs=[caption_output, motion_output, gr.HTML()]
    )

    gr.Markdown("### How it works:")
    gr.Markdown("1. Upload an image using the 'Upload Image' button.")
    gr.Markdown("2. Click 'Generate Caption' to get a description of the image.")
    gr.Markdown("3. If motion is detected, an animation will play.")

# Launch Gradio app
demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


In [23]:
with open('/home/nitin/Documents/Workspace/AI-Image-Generator/captions_train2017 (3).json', 'r') as f:
    data = json.load(f)
    data = data['annotations']

img_cap_pairs = []

for sample in data:
    img_name = '%012d.jpg' % sample['image_id']
    img_cap_pairs.append([img_name, sample['caption']])

captions = pd.DataFrame(img_cap_pairs, columns=['image', 'caption'])
# captions['image'] = captions['image'].apply(
#     lambda x: f'{BASE_PATH}/train2017/{x}'
# )
captions = captions.reset_index(drop=True)
captions.head()


,image,caption
0,000000203564.jpg,A bicycle replica with a clock as the front wh...
1,000000322141.jpg,A room with blue walls and a white sink and door.
2,000000016977.jpg,A car that seems to be parked illegally behind...
3,000000106140.jpg,A large passenger airplane flying through the ...
4,000000106140.jpg,There is a GOL plane taking off in a partly cl...


In [26]:
captions.tail()

,image,caption
591748,000000133071.jpg,a slice of bread is covered with a sour cream ...
591749,000000410182.jpg,A long plate hold some fries with some sliders...
591750,000000180285.jpg,Two women sit and pose with stuffed animals.
591751,000000133071.jpg,White Plate with a lot of guacamole and an ext...
591752,000000133071.jpg,A dinner plate has a lemon wedge garnishment.
